# Import data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir -p data/coco
!unrar x ./drive/MyDrive/artwork_captioning/data/datasets/coco/train2017.rar ./data/coco
!unrar x ./drive/MyDrive/artwork_captioning/data/datasets/coco/val2017.rar ./data/coco

Output streaming troncato alle ultime 5000 righe.
Extracting  ./data/coco/val2017/000000000285.jpg                           0%  OK 
Extracting  ./data/coco/val2017/000000000632.jpg                           0%  OK 
Extracting  ./data/coco/val2017/000000000724.jpg                           0%  OK 
Extracting  ./data/coco/val2017/000000000776.jpg                           0%  OK 
Extracting  ./data/coco/val2017/000000000785.jpg                           0%  OK 
Extracting  ./data/coco/val2017/000000000802.jpg                           0%  OK 
Extracting  ./data/coco/val2017/000000000872.jpg                           0%  OK 
Extracting  ./data/coco/val2017/000000000885.jpg                           0%  OK 
Extracting  ./data/coco/val2017/000000001000.jpg                           0%  OK 
Extracting  ./data/coco/val2017/000000001268.jpg                           0%  OK 
Extracting  ./data/coco/val201

In [ ]:
!unrar x ./drive/MyDrive/artwork_captioning/data/datasets/coco/annotations.rar ./data/coco


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from ./drive/MyDrive/artwork_captioning/data/datasets/coco/annotations.rar

Creating    ./data/coco/annotations                                   OK
Extracting  ./data/coco/annotations/captions_train2017.json                1%  3%  5%  6%  OK 
Extracting  ./data/coco/annotations/captions_val2017.json                  6%  OK 
Extracting  ./data/coco/annotations/instances_train2017.json               8% 10% 12% 13% 15% 17% 19% 21% 22% 24% 26% 28% 29% 31% 33% 35% 36% 38% 40% 42% 43% 45% 47% 49% 51% 52% 54% 56% 58% 59% 61% 63% 65% 66% 68%  OK 
Extracting  ./data/coco/annotations/instances_val2017.json                69% 70%  OK 
Extracting  ./data/coco/annotations/person_keypoints_train2017.json       72% 74% 75%

In [ ]:
!mkdir -p data/virtual_paintings/styleflow
!unrar x \
    ./drive/MyDrive/artwork_captioning/data/datasets/virtual_paintings/styleflow/train2017.rar \
    ./data/virtual_paintings/styleflow
!unrar x \
    ./drive/MyDrive/artwork_captioning/data/datasets/virtual_paintings/styleflow/val2017.rar \
    ./data/virtual_paintings/styleflow

Output streaming troncato alle ultime 5000 righe.
Extracting  ./data/virtual_paintings/styleflow/val2017/000000000285.jpg       0%  OK 
Extracting  ./data/virtual_paintings/styleflow/val2017/000000000632.jpg       0%  OK 
Extracting  ./data/virtual_paintings/styleflow/val2017/000000000724.jpg       0%  OK 
Extracting  ./data/virtual_paintings/styleflow/val2017/000000000776.jpg       0%  OK 
Extracting  ./data/virtual_paintings/styleflow/val2017/000000000785.jpg       0%  OK 
Extracting  ./data/virtual_paintings/styleflow/val2017/000000000802.jpg       0%  OK 
Extracting  ./data/virtual_paintings/styleflow/val2017/000000000872.jpg       0%  OK 
Extracting  ./data/virtual_paintings/styleflow/val2017/000000000885.jpg       0%  OK 
Extracting  ./data/virtual_paintings/styleflow/val2017/000000001000.jpg       0%  OK 
Extracting  ./data/virtual_paintings/styleflow/val2017/000000001268.jpg       0%  OK 


# Setup python environment

In [ ]:
!python -m pip install pyyaml==5.1
!pip install 'git+https://github.com/facebookresearch/detectron2.git@5aeb252b194b93dc2879b4ac34bc51a31b5aee13'

In [ ]:
!git clone https://github.com/rbarile17/artwork_captioning.git

Cloning into 'artwork_captioning'...
remote: Enumerating objects: 141, done.
remote: Counting objects: 100% (141/141), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 141 (delta 65), reused 108 (delta 37), pack-reused 0
Receiving objects: 100% (141/141), 1.21 MiB | 10.22 MiB/s, done.
Resolving deltas: 100% (65/65), done.


# Set models and dataloaders

In [ ]:
import os
os.environ['DETECTRON2_DATASETS'] = './data'

In [ ]:
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2
import torch
from pprint import pprint
import numpy as np
import random

In [ ]:
from detectron2 import model_zoo
from detectron2.config import get_cfg
from artwork_captioning.src.modules.model import VirtualPaintingsGeneralizedRCNN

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")

cfg.MODEL['META_ARCHITECTURE'] = 'VirtualPaintingsGeneralizedRCNN'
cfg.INPUT['RANDOM_FLIP'] = 'none'
cfg.INPUT.MIN_SIZE_TRAIN = (640, 672, 704, 736, 768)

In [ ]:
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer

coco_model = build_model(cfg)
DetectionCheckpointer(coco_model).load(cfg.MODEL.WEIGHTS)

print('COCO model loaded')

model_final_f6e8b1.pkl: 243MB [00:04, 55.2MB/s]                           


COCO model loaded


In [ ]:
torch.manual_seed(0)

import random
random.seed(0)

import numpy as np
np.random.seed(0)

In [ ]:
from artwork_captioning.src.modules.dataset_loading import coco_with_virtual_paintings

# dataset = DatasetCatalog.get("coco_2017_train_resized")
# dataset = DatasetCatalog.get("coco_2017_train_resized_no_augs")
dataset = coco_with_virtual_paintings('./data/virtual_paintings/styleflow/train2017')

In [ ]:
from detectron2.data import build_detection_train_loader, DatasetMapper
from artwork_captioning.src.modules.dataset_loading import CustomMapperWithAugmentations

dataloader = build_detection_train_loader(dataset, mapper=CustomMapperWithAugmentations(cfg, is_train=True), total_batch_size=4)

In [ ]:
from artwork_captioning.src.modules.dataset_loading import coco_with_virtual_paintings

# val_dataset = DatasetCatalog.get("coco_2017_val_resized")
# val_dataset = DatasetCatalog.get("coco_2017_val_resized_no_augs")
val_dataset = coco_with_virtual_paintings('./data/virtual_paintings/styleflow/val2017', False)

In [ ]:
from detectron2.data import build_detection_test_loader

val_dataloader = build_detection_test_loader(val_dataset, mapper=CustomMapperWithAugmentations(cfg, is_train=True))

In [ ]:
def get_image_features(model, batch, image_type):
    model.image_type = image_type
    images = model.preprocess_image(batch)
    features = model.backbone(images.tensor)

    return features

In [ ]:
from detectron2.utils.events import EventStorage

def object_detection(model, batch):
    model.image_type = 'virtual_painting'
    return model(batch)

In [ ]:
virtual_paintings_model = build_model(cfg)
DetectionCheckpointer(virtual_paintings_model).load(cfg.MODEL.WEIGHTS)

for param in coco_model.parameters():
    param.requires_grad = False

for param in virtual_paintings_model.parameters():
    param.requires_grad = False

for param in virtual_paintings_model.backbone.parameters():
    param.requires_grad = True

virtual_paintings_model.to('cuda:0')
print('model loaded')

model loaded


# Training config

In [ ]:
BATCHES_NUMBER = int(len(dataset) / dataloader.batch_size)
EPOCHS_NUMBER = 10
DECAY_EPOCHS = 3

cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.005
cfg.SOLVER.WEIGHT_DECAY = 0.0005
cfg.SOLVER.MOMENTUM = 0.9
cfg.SOLVER.GAMMA = 0.1
cfg.SOLVER.MAX_ITER = BATCHES_NUMBER * EPOCHS_NUMBER
cfg.SOLVER.STEPS = (BATCHES_NUMBER * DECAY_EPOCHS, BATCHES_NUMBER * DECAY_EPOCHS * 2, BATCHES_NUMBER * DECAY_EPOCHS * 3)
cfg.SOLVER.WARMUP_FACTOR = 1.0 / 5000
cfg.SOLVER.WARMUP_ITERS = 5000
cfg.SOLVER.WARMUP_METHOD = "linear"

cfg.SOLVER.CHECKPOINT_PERIOD = BATCHES_NUMBER
cfg.TEST.EVAL_PERIOD = BATCHES_NUMBER

cfg.OUTPUT_DIR = './drive/MyDrive/artwork_captioning/object_detection_output/styleflow'

In [ ]:
from collections import OrderedDict
from detectron2.data import MetadataCatalog
from detectron2.evaluation import COCOEvaluator, DatasetEvaluators, inference_on_dataset

def do_test(cfg, model):
    model.image_type = 'virtual_painting'
    for dataset_name in cfg.DATASETS.TEST:
        results = inference_on_dataset(model, val_dataloader, COCOEvaluator(dataset_name, output_dir=None))
    return results

In [ ]:
from detectron2.solver import build_lr_scheduler, build_optimizer
from detectron2.checkpoint import PeriodicCheckpointer
from detectron2.engine import default_writers

import detectron2.utils.comm as comm

import json

DETECTION_LOSSES_NUMBER = 4
FEATURES_NUMBER = 5

def do_train(resume=False):
    virtual_paintings_model.evaluation = False
    virtual_paintings_model.train()
    optimizer = build_optimizer(cfg, virtual_paintings_model)
    scheduler = build_lr_scheduler(cfg, optimizer)

    checkpointer = DetectionCheckpointer(virtual_paintings_model, cfg.OUTPUT_DIR, optimizer=optimizer, scheduler=scheduler)

    start_iter = (checkpointer.resume_or_load(cfg.MODEL.WEIGHTS, resume=resume).get("iteration", -1) + 1)
    print(f'start iter: {start_iter}')
    max_iter = cfg.SOLVER.MAX_ITER
    running_loss = 0
    epoch = 9

    writers = default_writers(cfg.OUTPUT_DIR, max_iter) 
    
    periodic_checkpointer = PeriodicCheckpointer(checkpointer, cfg.SOLVER.CHECKPOINT_PERIOD, max_iter=max_iter)

    align_loss_fn = torch.nn.MSELoss()
    with EventStorage(start_iter) as storage:
        with tqdm(desc=f'Train', unit='iteration', total=max_iter - start_iter) as pbar:
            for batch, iteration in zip(dataloader, range(start_iter, max_iter)):
                storage.iter = iteration

                virtual_painting_features_dict = get_image_features(virtual_paintings_model, batch, 'virtual_painting')
                features_dict = get_image_features(coco_model, batch, 'image')

                align_loss = 0
                for virtual_painting_features, features in zip(virtual_painting_features_dict.values(), features_dict.values()):
                    align_loss += align_loss_fn(virtual_painting_features, features.detach().clone())

                align_loss = align_loss / FEATURES_NUMBER

                detection_loss_dict = object_detection(virtual_paintings_model, batch)
                detection_loss = sum(detection_loss_dict.values()) / DETECTION_LOSSES_NUMBER

                total_loss = align_loss + detection_loss

                detection_loss_dict_reduced = {k: v.item() for k, v in comm.reduce_dict(detection_loss_dict).items()}
                detection_loss_reduced = sum(loss for loss in detection_loss_dict_reduced.values()) / DETECTION_LOSSES_NUMBER
                total_loss_reduced = align_loss.item() + detection_loss_reduced
                storage.put_scalars(total_loss=total_loss_reduced)
                
                optimizer.zero_grad()
                total_loss.backward()
                optimizer.step()
                storage.put_scalar("lr", optimizer.param_groups[0]["lr"], smoothing_hint=False)

                this_loss = total_loss.item()
                running_loss += this_loss

                pbar.set_postfix(
                    epoch = epoch,
                    loss = running_loss / (iteration - start_iter + 1),
                    lr = f"{optimizer.param_groups[0]['lr']:.7f}")
                pbar.update()

                scheduler.step()

                if iteration - start_iter > 5 and ((iteration + 1) % 20 == 0 or iteration == max_iter - 1):
                    for default_writer in writers:
                        default_writer.write()

                periodic_checkpointer.step(iteration)

                if (cfg.TEST.EVAL_PERIOD > 0 
                    and (iteration + 1) % cfg.TEST.EVAL_PERIOD == 0 
                    and iteration != max_iter - 1):

                    with open(f'{cfg.OUTPUT_DIR}/val_metrics.json', 'r') as f:
                        val_metrics = json.load(f)
                    val_metrics.update({epoch: do_test(cfg, virtual_paintings_model)})    
                    with open(f'{cfg.OUTPUT_DIR}/val_metrics.json', 'w') as f:
                        json.dump(val_metrics, f)

                    epoch += 1

# Training

In [ ]:
# with open(f'{cfg.OUTPUT_DIR}/val_metrics.json', 'w') as f:
#     json.dump({}, f)

do_train(True)

Train: 100%|██████████| 29571/29571 [4:36:39<00:00,  1.78iteration/s, epoch=9, loss=1.18, lr=0.0000050]
